In [1]:
import pandas as pd
import re
import string
import nltk
import spacy

2023-05-04 12:29:31.769851: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-04 12:29:33.199596: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 12:29:35.966213: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Book title

전처리 이후 train_data['Book-Title'].nunique() >>> 205946

In [6]:
# 필요한 컴포넌트를 다운로드합니다.
nltk.download("stopwords")
nltk.download("wordnet")

# NLTK의 불용어 리스트를 불러옵니다.
from nltk.corpus import stopwords

# spaCy의 영어 NLP 모델을 로드합니다.
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/conceptelling/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/conceptelling/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# 예제 데이터셋을 생성합니다.
train_data = pd.read_csv("./data/test.csv")
df = pd.DataFrame(train_data[["Book-ID","Book-Title"]])

In [8]:
# 소문자 변환 함수
def to_lowercase(text):
    return text.lower()

# 특수 문자 확인 함수
def check_special_chars(text):
    return set(re.findall(r"[{}]".format(string.punctuation), text))

# 특수 문자 제거 함수
def remove_special_chars(text):
    return re.sub(r"[{}]".format(string.punctuation), "", text)

# 불용어 제거 함수
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word not in stop_words])

# 표제어 추출 함수
def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

In [9]:
# 데이터 프레임에 적용
df["Book-Title_lower"] = df["Book-Title"].apply(to_lowercase)
df["Book-Title_special_chars"] = df["Book-Title_lower"].apply(check_special_chars)
df["Book-Title_no_special_chars"] = df["Book-Title_lower"].apply(remove_special_chars)
df["Book-Title_no_stopwords"] = df["Book-Title_no_special_chars"].apply(remove_stopwords)
df["Book-Title_lemmatized"] = df["Book-Title_no_stopwords"].apply(lemmatize)

In [11]:
bookTitle_prePro_selected = df[["Book-Title_lemmatized"]]
bookTitle_prePro_selected.rename(columns={'Book-Title_lemmatized': 'Book-Title'}, inplace=True)
train_data.update(bookTitle_prePro_selected)

/home/conceptelling/miniconda3/lib/python3.10/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [12]:
train_data

,ID,User-ID,Book-ID,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,TEST_000000,USER_00008,BOOK_047966,37.0,"vermilion, ohio, usa",bird prey novel suspense,J.A. Jance,2002.0,Avon
1,TEST_000001,USER_00008,BOOK_119494,37.0,"vermilion, ohio, usa",midnight voice,JOHN SAUL,2003.0,Ballantine Books
2,TEST_000002,USER_00008,BOOK_151775,37.0,"vermilion, ohio, usa",break free prescription personal organizationa...,David M. Noer,1996.0,Jossey-Bass
3,TEST_000003,USER_00008,BOOK_176255,37.0,"vermilion, ohio, usa",bitter harvest,Ann Rule,1999.0,Pocket
4,TEST_000004,USER_00008,BOOK_187307,37.0,"vermilion, ohio, usa",embrace light,Betty J. Eadie,1994.0,Bantam Books
...,...,...,...,...,...,...,...,...,...
159616,TEST_159616,USER_92086,BOOK_159050,0.0,"mountain view, california, usa",fat flush plan,Ann Louise Gittleman,2001.0,McGraw-Hill/Contemporary Books
159617,TEST_159617,USER_92086,BOOK_196481,0.0,"mountain view, california, usa",die alone wwii epic escape endurance,David Howarth,1999.0,The Lyons Press
159618,TEST_159618,USER_92086,BOOK_199754,0.0,"mountain view, california, usa",60 yard master golfs short game,Raymond Floyd,1992.0,Perennial
159619,TEST_159619,USER_92086,BOOK_227481,0.0,"mountain view, california, usa",solo adventure,Susan Fox Rogers,1996.0,Seal Press (WA)


word2vec 임베딩으로 변환. 다만 이 작업은 전략적으로 고려했을 때, Book Node ID로 Feature matrix 정렬이 되고 난 다음이 유리함.
* 따라서 여기서 중단!

# Location

In [17]:
import pickle

# 마지막 나라만 가져온다.
train_data['Location'] = train_data['Location'].apply(lambda x: x.split(', ')[-1].strip())

with open("./data/country_name_mapping.pkl", "rb") as f:
    country_name_mapping = pickle.load(f)
    
# 'location' 열의 값을 국가 이름으로 변경
train_data['Location'] = train_data['Location'].apply(lambda x: country_name_mapping.get(x, x))

# Age, Year-Of-Publication

In [18]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max 스케일링을 적용할 피처 선택
features_to_scale = ['Age', 'Year-Of-Publication']

# MinMaxScaler 객체 생성
scaler = MinMaxScaler()

# Min-Max 스케일링 적용
train_data[features_to_scale] = scaler.fit_transform(train_data[features_to_scale])

# Book-Author
Publisher_list = train_data["Book-Author"].unique().tolist() >>> 92,635     
지나치게 sparse

# Publisher

In [19]:
# remove special characters and convert to lowercase
train_data['Publisher'] =train_data['Publisher'].str.replace('[^a-zA-Z0-9\s]+', '').str.lower()

# remove leading and trailing whitespaces
train_data['Publisher'] = train_data['Publisher'].str.strip()

# get the number of unique publishers
# num_publishers = train_data['Publisher'].nunique()

/tmp/ipykernel_2778/2517125782.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['Publisher'] =train_data['Publisher'].str.replace('[^a-zA-Z0-9\s]+', '').str.lower()


In [20]:
train_data['Publisher'].nunique()

6420

In [21]:
top_publishers = train_data['Publisher'].value_counts()

In [22]:
top_publishers.head(3000) #5권 이하의 책을 보유한 출판사는 전부 날려버립니다!

ballantine books                       5028
berkley publishing group               4967
pocket                                 4777
warner books                           4237
harlequin                              3668
                                       ... 
mit press                                 2
scepter publishers inc                    2
hayden book co                            2
dundurn press                             2
elsevier science health science div       2
Name: Publisher, Length: 3000, dtype: int64

In [23]:
top_publishers.head(50)  # 유사한 출판사 이름 확인

ballantine books            5028
berkley publishing group    4967
pocket                      4777
warner books                4237
harlequin                   3668
bantam books                3554
signet book                 3137
bantam                      2777
penguin books               2734
avon                        2555
jove books                  2411
fawcett books               2275
silhouette                  2232
perennial                   2169
dell                        2101
harpertorch                 2025
scholastic                  1931
vintage books usa           1793
simon amp schuster          1603
harpercollins               1579
st martins press            1561
dell publishing company     1553
scribner                    1325
little brown                1321
tor books                   1231
penguin usa                 1137
mira                        1106
del rey books               1098
random house inc            1043
anchor                      1014
plume book

In [24]:
# 1 단어 이상 일치하면 같은 것으로 통합해버리는 map을 만들어 매핑.
# difflib 라이브러리를 사용할 수도 있지만, time cost가 너무 크다.

def create_publisher_map(publisher_list):
    publisher_map = {}
    for publisher in publisher_list:
        words = publisher.split()
        for word in words:
            if word not in publisher_map:
                publisher_map[word] = publisher
            else:
                if len(publisher) < len(publisher_map[word]):
                    publisher_map[word] = publisher
    return publisher_map

def map_publisher(publisher, publisher_map):
    words = publisher.split()
    for word in words:
        if word in publisher_map:
            return publisher_map[word]
    return publisher

In [25]:
unique_publishers = train_data['Publisher'].unique()
publisher_map = create_publisher_map(unique_publishers)
train_data['Publisher_cleaned'] = train_data['Publisher'].apply(lambda x: map_publisher(x, publisher_map))

In [26]:
train_data['Publisher_cleaned'].nunique()

3569

In [27]:
# 출판사 이름의 빈도수를 계산합니다.
publisher_counts = train_data['Publisher_cleaned'].value_counts()

# 빈도수가 2 이하인 출판사 이름을 찾습니다.
rare_publishers = publisher_counts[publisher_counts <= 2].index

# 빈도수가 2 이하인 출판사 이름을 'Other'로 변환합니다.
train_data['Publisher_cleaned'] = train_data['Publisher_cleaned'].apply(lambda x: 'Other' if x in rare_publishers else x)

In [28]:
train_data['Publisher_cleaned'].nunique()

1634

In [29]:
train_data['Publisher_cleaned'].value_counts().head(50)  # 정리된 것 확인.

bantam                   7855
pocket                   5773
penguin                  5473
berkley                  5304
warner                   5276
ballantine               5047
harlequin                3771
dell                     3723
st martins               3620
signet                   3494
avon                     3346
random house             3218
scholastic               2816
harpercollins            2790
vintage                  2733
fawcett                  2534
simon pulse              2483
jove                     2480
Other                    2476
perennial                2393
silhouette               2245
harpertorch              2025
little brown             1828
tor                      1711
del rey                  1694
anchor                   1529
scribner                 1446
mira                     1286
putnam                   1256
new press                1234
ace                      1024
plume                     978
doubleday                 898
hyperion  

In [30]:
train_data.drop(["Publisher"],axis=1,inplace=True)

In [31]:
train_data.rename(columns={'Publisher_cleaned': 'Publisher'}, inplace=True)

# Done!

In [32]:
train_data

,ID,User-ID,Book-ID,Age,Location,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,TEST_000000,USER_00008,BOOK_047966,0.156118,United States,bird prey novel suspense,J.A. Jance,0.990603,avon
1,TEST_000001,USER_00008,BOOK_119494,0.156118,United States,midnight voice,JOHN SAUL,0.991098,ballantine
2,TEST_000002,USER_00008,BOOK_151775,0.156118,United States,break free prescription personal organizationa...,David M. Noer,0.987636,josseybass
3,TEST_000003,USER_00008,BOOK_176255,0.156118,United States,bitter harvest,Ann Rule,0.989120,pocket
4,TEST_000004,USER_00008,BOOK_187307,0.156118,United States,embrace light,Betty J. Eadie,0.986647,bantam
...,...,...,...,...,...,...,...,...,...
159616,TEST_159616,USER_92086,BOOK_159050,0.000000,United States,fat flush plan,Ann Louise Gittleman,0.990109,mcgrawhillcontemporary
159617,TEST_159617,USER_92086,BOOK_196481,0.000000,United States,die alone wwii epic escape endurance,David Howarth,0.989120,the museum
159618,TEST_159618,USER_92086,BOOK_199754,0.000000,United States,60 yard master golfs short game,Raymond Floyd,0.985658,perennial
159619,TEST_159619,USER_92086,BOOK_227481,0.000000,United States,solo adventure,Susan Fox Rogers,0.987636,seal press


In [33]:
train_data.to_csv("prepro_test_data.csv")